In [ ]:
%load_ext autoreload
%autoreload 2

# Super Weights Analysis - Identifying Super Weights and their Impact on Model Performance

In [ ]:
from research.researcher import SuperWeightResearchSession
import torch

In [4]:
model_name = 'allenai/OLMo-1B-0724-hf'

In [5]:
session = SuperWeightResearchSession.from_model_name(model_name)

/Users/fabiangrob/miniforge3/envs/super-weights/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/fabiangrob/miniforge3/envs/super-weights/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/fabiangrob/miniforge3/envs/super-weights/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/fabiangrob/miniforge3/envs/super-weights/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/fabiangrob/miniforge3/envs/super-weights/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/fabiangrob/miniforge3/envs/super-weights/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/fabiangrob/miniforge3/envs/super-weights/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functio

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-06-26 01:36:39,071 - SuperWeightDetector_13509139552 - INFO - SuperWeightDetector initialized for allenai-OLMo-1B-0724-hf
2025-06-26 01:36:39,071 - SuperWeightDetector_13509139552 - INFO - Model has 16 layers
2025-06-26 01:36:39,071 - SuperWeightResearch_4587844080 - INFO - Detected standard transformer architecture
2025-06-26 01:36:39,072 - SuperWeightDetector_13509140896 - INFO - SuperWeightDetector initialized for allenai-OLMo-1B-0724-hf
2025-06-26 01:36:39,072 - SuperWeightDetector_13509140896 - INFO - Model has 16 layers
2025-06-26 01:36:39,073 - SuperWeightResearch_4587844080 - INFO - SuperWeightResearchSession initialized
2025-06-26 01:36:39,073 - SuperWeightResearch_4587844080 - INFO - Using detector: SuperWeightDetector
2025-06-26 01:36:39,071 - SuperWeightDetector_13509139552 - INFO - Model has 16 layers
2025-06-26 01:36:39,071 - SuperWeightResearch_4587844080 - INFO - Detected standard transformer architecture
2025-06-26 01:36:39,072 - SuperWeightDetector_13509140896 - 

In [6]:
sw = session.detect_super_weights(spike_threshold=70.0)

2025-06-26 01:36:40,898 - SuperWeightResearch_4587844080 - INFO - Starting super weight detection
2025-06-26 01:36:40,898 - SuperWeightDetector_13509140896 - INFO - Starting super weight detection
2025-06-26 01:36:40,898 - SuperWeightDetector_13509140896 - INFO - Parameters: threshold=70.0, max_iterations=10
2025-06-26 01:36:40,903 - SuperWeightDetector_13509140896 - INFO - === Iteration 1 ===
2025-06-26 01:36:40,898 - SuperWeightDetector_13509140896 - INFO - Starting super weight detection
2025-06-26 01:36:40,898 - SuperWeightDetector_13509140896 - INFO - Parameters: threshold=70.0, max_iterations=10
2025-06-26 01:36:40,903 - SuperWeightDetector_13509140896 - INFO - === Iteration 1 ===
2025-06-26 01:36:41,254 - SuperWeightDetector_13509140896 - INFO - Found 2 potential super weights in iteration 1
2025-06-26 01:36:41,254 - SuperWeightDetector_13509140896 - INFO - Found 2 new super weights:
2025-06-26 01:36:41,255 - SuperWeightDetector_13509140896 - INFO -   1. Layer 1 mlp.down_proj.we

In [7]:
sw

[SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00),
 SuperWeight(layer=15, coords=[1764, 6840], input=-283.00, output=415.00)]

## Vocabulary analysis

In [18]:
sophisticated_sw = sw[0]

In [9]:
vocab_analysis = session.analyzer.vocabulary_analyzer.analyze_vocabulary_effects(sophisticated_sw, n_samples=500)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-06-25 13:33:48,681 - SuperWeightManager_13627786576 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-25 13:34:13,472 - SuperWeightManager_13627786576 - INFO - Restored 1/1 weights


In [10]:
session.analyzer.vocabulary_analyzer.display_analysis_results(vocab_analysis, top_k=50)


=== Vocabulary Analysis: Layer 1 mlp.down_proj.weight[1764, 1710] ===

Effect Statistics:
  Mean: -0.0568
  Std: 1.0967
  Kurtosis: 0.4625
  Skew: 0.6961
  Significant effects: 17426

Classification:
  Type: PARTITION
  Description: Affects broad token classes (boost some, suppress others)
  Confidence: 0.30

Top 50 Boosted Tokens:
   1. ' Ac'                (effect: +4.562)
   2. ' Three'             (effect: +4.535)
   3. ' Question'          (effect: +4.473)
   4. ' Two'               (effect: +4.406)
   5. ' Ind'               (effect: +4.395)
   6. ' Du'                (effect: +4.379)
   7. ' Witt'              (effect: +4.254)
   8. ' Dro'               (effect: +4.246)
   9. ' In'                (effect: +4.223)
  10. ' Private'           (effect: +4.211)
  11. ' Wat'               (effect: +4.191)
  12. ' Options'           (effect: +4.184)
  13. ' Tri'               (effect: +4.172)
  14. ' Modern'            (effect: +4.164)
  15. ' Models'            (effect: +4.145)
  16.

In [11]:
# analyze w_u @ w_out, full neuron with super weight
neuron_analysis = session.analyzer.vocabulary_analyzer.analyze_neuron_vocabulary_effects(sophisticated_sw)

In [12]:
session.analyzer.vocabulary_analyzer.display_analysis_results(neuron_analysis, top_k=20)


=== Vocabulary Analysis: Layer 1 mlp.down_proj.weight[1764, 1710] ===

Effect Statistics:
  Mean: -0.0002
  Std: 0.0067
  Kurtosis: -3.0000
  Skew: 0.1986
  Significant effects: 0

Classification:
  Type: UNCLEAR
  Description: Effects too small or distributed to classify clearly
  Confidence: 0.30

Top 20 Boosted Tokens:
   1. ' �'                 (effect: +0.038)
   2. ' vet'               (effect: +0.031)
   3. ' stir'              (effect: +0.031)
   4. ' appeal'            (effect: +0.031)
   5. ' appro'             (effect: +0.031)
   6. ' comm'              (effect: +0.031)
   7. ' Polish'            (effect: +0.030)
   8. ' plac'              (effect: +0.030)
   9. ' bro'               (effect: +0.029)
  10. ' MOT'               (effect: +0.029)
  11. ' aid'               (effect: +0.029)
  12. ' fully'             (effect: +0.028)
  13. ' mac'               (effect: +0.028)
  14. '”?'                 (effect: +0.028)
  15. ' critic'            (effect: +0.028)
  16. ' wash'  

## Vocabulary analysis with intermediate activations

In [ ]:
# Basic cascade analysis
cascade_results = session.analyzer.vocabulary_analyzer.analyze_activation_cascade(
    sw[0], 
    input_text="Apple Inc. is a technology company."
)

Capturing baseline activations...
Capturing modified activations...


2025-06-26 19:35:39,061 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-26 19:35:39,213 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Projecting activations through 16 layers...
  Processing layer 15/15


In [30]:
# Display results
session.analyzer.vocabulary_analyzer.display_cascade_analysis(cascade_results)


CASCADE ANALYSIS (FULL_PROJECTION): Layer 1 mlp.down_proj.weight[1764, 1710]

📊 SUMMARY:
  Analysis type: full_projection
  Layers analyzed: 16
  Peak effect at layer: 8
  Final magnitude: 213.6250
  Effect evolution: increasing

🔄 PROPAGATION ANALYSIS:
  Pattern: amplifying
  Amplification ratio: 0.000
  Peak layer: 8
  Critical layers: [1, 2]

🎯 CONVERGENCE ANALYSIS:
  Convergence layer: 15
  Final similarity: 1.000
  Stable tokens: ' incorporated', ' head', ' publicly', ' privately', ' profitable'

📈 KEY LAYERS ANALYSIS:

  Layer 0:
    Effect magnitude: 0.0000
    Activation magnitude: 0.0000
    Layers remaining: 15
    Top boosted: '|||IP_ADDRESS|||'(+0.00), '<|padding|>'(+0.00), '!'(+0.00), '"'(+0.00), '#'(+0.00)

  Layer 8:
    Effect magnitude: 235.3750
    Activation magnitude: 3.7246
    Layers remaining: 7
    Top boosted: ' founded'(+7.30), ' owned'(+6.64), ' created'(+6.32), ' governed'(+6.15), 'lik'(+5.74)

  Layer 15:
    Effect magnitude: 213.6250
    Activation magni

In [45]:
input_text = "The company is great. The company is"
# Tokenize input text
tokens = session.tokenizer(input_text, return_tensors='pt').to(session.model.device)

In [46]:
tokens

{'input_ids': tensor([[ 510, 2567,  310, 1270,   15,  380, 2567,  310]], device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='mps:0')}

In [47]:
output = session.model.generate(
    tokens['input_ids'],
    max_new_tokens=32,
    do_sample=False,
    temperature=0.0
)
# Decode output
output_text = session.tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

The company is great. The company is great. The company is great. The company is great. The company is great. The company is great. The company is great. The company is great.


In [48]:
# now with zeroed super weight
with session.manager.temporary_scale([sw[0]], scale_factor=-1.0):
    output_zeroed = session.model.generate(
        tokens['input_ids'],
        max_new_tokens=32,
        do_sample=False,
        temperature=0.0
    )
    # Decode output
    output_text_zeroed = session.tokenizer.decode(output_zeroed[0], skip_special_tokens=True)
    print(output_text_zeroed)

2025-06-26 19:42:52,323 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by -1.000
2025-06-26 19:42:53,458 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


The company is great. The company is", the, the, the, the, the, the, the, the, the, the, the, the, the, the, the,),


In [54]:
scale_factor = -100.0

print(f"Scale factor for inverted super weight: {scale_factor}")

test_prompts = [
    # Different starting tokens
    "Apple Inc. is a technology company. Apple",
    "Microsoft Corporation develops software. Microsoft", 
    "The cat sat on the mat. The",
    "John walked to the store. John",
    "Beautiful flowers bloom in spring. Beautiful",
    
    # Different sentence structures
    "Yesterday I went to the park. Yesterday",
    "Quickly running down the street, he",
    "In the morning, coffee tastes best. In",
    "After finishing work, she decided to",
    "Before starting the meeting, everyone should",
    
    # Different content domains
    "The scientific method requires careful observation. The",
    "Economic indicators suggest market volatility. Economic",
    "Political leaders gathered for summit talks. Political",
    "Artificial intelligence transforms modern computing. Artificial",
    "Climate change affects global weather patterns. Climate",
    
    # Punctuation and formatting
    "Hello, world! This is a test. Hello,",
    "Question: What is the capital? Question:",
    "First, second, third item on list. First,",
    "Note: Please review carefully. Note:",
    
    # Numbers and special cases
    "In 1995, the internet was growing. In",
    "Chapter 7 discusses advanced topics. Chapter",
    "Version 2.0 includes new features. Version",
    
    # Function words
    "And then something interesting happened. And",
    "But there was a problem. But",
    "Or maybe we should try differently. Or",
    "So the conclusion is clear. So"
]

# Test code
for i, prompt in enumerate(test_prompts):
    print(f"\n--- Test {i+1}: {prompt} ---")
    
    tokens = session.tokenizer(prompt, return_tensors='pt').to(session.model.device)
    
    # Normal generation
    with torch.no_grad():
        normal_output = session.model.generate(
            tokens['input_ids'],
            max_new_tokens=16,
            do_sample=False,
            temperature=0.0
        )
    normal_text = session.tokenizer.decode(normal_output[0], skip_special_tokens=True)
    
    # Inverted super weight generation
    with session.manager.temporary_scale([sw[0]], scale_factor=scale_factor):
        with torch.no_grad():
            inverted_output = session.model.generate(
                tokens['input_ids'],
                max_new_tokens=16,
                do_sample=False,
                temperature=0.0
            )
    inverted_text = session.tokenizer.decode(inverted_output[0], skip_special_tokens=True)
    
    print(f"Normal:   {normal_text}")
    print(f"Inverted: {inverted_text}")

Scale factor for inverted super weight: -100.0

--- Test 1: Apple Inc. is a technology company. Apple ---


/Users/fabiangrob/miniforge3/envs/super-weights/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
2025-06-26 20:01:31,881 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by -100.000
2025-06-26 20:01:32,461 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Normal:   Apple Inc. is a technology company. Apple Inc. is a technology company. Apple Inc. is a technology company. Apple
Inverted: Apple Inc. is a technology company. Apple), a. a. a. a. a. a. a. a

--- Test 2: Microsoft Corporation develops software. Microsoft ---


2025-06-26 20:01:33,038 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by -100.000
2025-06-26 20:01:33,531 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Normal:   Microsoft Corporation develops software. Microsoft Corporation is a multinational corporation headquartered in Redmond, Washington, United
Inverted: Microsoft Corporation develops software. Microsoft Windows Windows Windows Windows Windows Windows Windows Windows Windows Windows Windows Windows Windows Windows Windows Windows

--- Test 3: The cat sat on the mat. The ---


...

Normal:   But there was a problem. But it was a problem that I could solve.

I had a friend who
Inverted: But there was a problem. But a', a, a, a, a, a, a, a,

--- Test 25: Or maybe we should try differently. Or ---


2025-06-26 20:01:56,496 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by -100.000
2025-06-26 20:01:57,029 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Normal:   Or maybe we should try differently. Or maybe we should try to be more like the people we want to be.

Inverted: Or maybe we should try differently. Or a. a. a. a. a. a. a. a.

--- Test 26: So the conclusion is clear. So ---


2025-06-26 20:01:57,569 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by -100.000
2025-06-26 20:01:58,114 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Normal:   So the conclusion is clear. So, what is the answer?

A:
the answer is yes.
Inverted: So the conclusion is clear. So..., the........................................................


In [58]:
scale_factor = -1.0

print(f"Scale factor for inverted super weight: {scale_factor}")

test_prompts = [
    "Very very very unusual word. Very",  # Force rare context
    "The the the common phrase. The",     # Force common context
    "The doctor examined the patient. The doctor",
    "The programmer coded the function. The programmer", 
    "The chef prepared the meal. The chef"
]

# Test code
for i, prompt in enumerate(test_prompts):
    print(f"\n--- Test {i+1}: {prompt} ---")
    
    tokens = session.tokenizer(prompt, return_tensors='pt').to(session.model.device)
    
    # Normal generation
    with torch.no_grad():
        normal_output = session.model.generate(
            tokens['input_ids'],
            max_new_tokens=16,
            do_sample=False,
            temperature=0.0
        )
    normal_text = session.tokenizer.decode(normal_output[0], skip_special_tokens=True)
    
    # Inverted super weight generation
    with session.manager.temporary_scale([sw[0]], scale_factor=scale_factor):
        with torch.no_grad():
            inverted_output = session.model.generate(
                tokens['input_ids'],
                max_new_tokens=16,
                do_sample=False,
                temperature=0.0
            )
    inverted_text = session.tokenizer.decode(inverted_output[0], skip_special_tokens=True)
    
    print(f"Normal:   {normal_text}")
    print(f"Inverted: {inverted_text}")

Scale factor for inverted super weight: -1.0

--- Test 1: Very very very unusual word. Very ---


2025-06-26 20:07:51,173 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by -1.000
2025-06-26 20:07:51,670 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Normal:   Very very very unusual word. Very very very unusual word.

I'm not sure if it's a word
Inverted: Very very very unusual word. Very
”?                            

--- Test 2: The the the common phrase. The ---


2025-06-26 20:07:52,167 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by -1.000
2025-06-26 20:07:52,482 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Normal:   The the the common phrase. The common phrase is the common phrase.

The common phrase is the common phrase
Inverted: The the the common phrase. The common phrase. The common phrase.
”?

--- Test 3: The doctor examined the patient. The doctor ---


...

Normal:   The doctor examined the patient. The doctor said, "You have a very serious disease. You need to have an operation
Inverted: The doctor examined the patient. The doctor, in", the, in, a, in, a, a, a

--- Test 4: The programmer coded the function. The programmer ---


2025-06-26 20:07:53,993 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by -1.000
2025-06-26 20:07:54,500 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Normal:   The programmer coded the function. The programmer is the one who knows the code. The programmer is the one who knows the
Inverted: The programmer coded the function. The programmer, the", the, the, the, the, the, the, the

--- Test 5: The chef prepared the meal. The chef ---


2025-06-26 20:07:54,850 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by -1.000
2025-06-26 20:07:55,355 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Normal:   The chef prepared the meal. The chef was a good cook.
A:
B
Inverted: The chef prepared the meal. The chef, a," the," the, the, the, the, the, the


In [31]:
cascade_results

{'analysis_type': 'full_projection',
 'super_weight': SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00),
 'input_text': 'The company. The company is',
 'cascade_effects': {0: {'activation_magnitude': 0.0,
   'vocab_effect': array([0., 0., 0., ..., 0., 0., 0.], dtype=float16),
   'statistics': {'mean': 0.0,
    'std': 0.0,
    'variance': 0.0,
    'kurtosis': nan,
    'skew': nan,
    'max_effect': 0.0,
    'min_effect': 0.0,
    'num_positive': 0,
    'num_negative': 0,
    'num_significant': 0},
   'top_tokens': {'top_boosted': [{'token_id': 0,
      'token_str': '|||IP_ADDRESS|||',
      'effect_magnitude': 0.0},
     {'token_id': 1, 'token_str': '<|padding|>', 'effect_magnitude': 0.0},
     {'token_id': 2, 'token_str': '!', 'effect_magnitude': 0.0},
     {'token_id': 3, 'token_str': '"', 'effect_magnitude': 0.0},
     {'token_id': 4, 'token_str': '#', 'effect_magnitude': 0.0},
     {'token_id': 5, 'token_str': '$', 'effect_magnitude': 0.0},
     {'token_id': 6

In [25]:
# Compare both methods
comparison = session.analyzer.vocabulary_analyzer.compare_cascade_methods(sw[0])
session.analyzer.vocabulary_analyzer.display_cascade_comparison(comparison)

Running cascade method comparison for Layer 1 mlp.down_proj.weight[1764, 1710]...
Running cascade analysis using full_projection method...
Capturing baseline activations...
Capturing modified activations...


2025-06-26 19:30:59,784 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-26 19:30:59,866 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Projecting activations through 16 layers...
  Processing layer 15/15
Running cascade analysis using residual_stream method...
Capturing residual stream...
Capturing residual stream using universal layer capture...


2025-06-26 19:31:00,303 - SuperWeightManager_13509139552 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-26 19:31:00,391 - SuperWeightManager_13509139552 - INFO - Restored 1/1 weights


Capturing residual stream using universal layer capture...
Analyzing residual differences across 16 layers...
  Processing layer 15/15

CASCADE METHODS COMPARISON: Layer 1 mlp.down_proj.weight[1764, 1710]

📊 COMPARISON METRICS:
  Final effects correlation: 0.755
  Final effects cosine similarity: 0.755
  Magnitude trajectory correlation: 0.580
  Full projection final magnitude: 283.2500
  Residual stream final magnitude: 236.0921

🎯 SUMMARY:
  Methods agree: True
  Preferred method: full_projection
  Consistency score: 0.667

💡 INTERPRETATION:
  ✅ Both methods show consistent results
  → Super weight effects are stable across analysis approaches

------------------------------------------------------------


In [16]:
# Examine how vocabulary effects change at each layer
residual_effects = cascade_results['residual_effects']

for layer_idx in sorted(residual_effects.keys()):
    layer_data = residual_effects[layer_idx]
    print(f"\nLayer {layer_idx}:")
    print(f"  Direct effect magnitude: {layer_data['effect_magnitude']:.4f}")
    print(f"  Cumulative magnitude: {layer_data['cumulative_magnitude']:.4f}")
    print(f"  Amplification factor: {layer_data['amplification_factor']:.3f}")
    
    # Look at top affected tokens at this layer
    top_tokens = layer_data['top_tokens']['top_boosted'][:5]
    for token in top_tokens:
        print(f"    {token['token_str']}: {token['effect_magnitude']:+.3f}")

KeyError: 'residual_effects'

## Perplexity analysis

In [8]:
perplexity_impact = session.analyzer.metrics_analyzer.measure_perplexity_impact(sophisticated_sw, n_samples=500)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-06-10 01:15:36,807 - SuperWeightManager_13442570496 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-10 01:16:41,770 - SuperWeightManager_13442570496 - INFO - Restored 1/1 weights


In [9]:
perplexity_impact

{'super_weight': SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00),
 'baseline_perplexity': 17.51461410522461,
 'modified_perplexity': 9591.20703125,
 'perplexity_ratio': 547.6116672412978,
 'perplexity_increase': 9573.692417144775,
 'impact_severity': 'catastrophic',
 'dataset_info': {'name': 'wikitext',
  'config': 'wikitext-2-raw-v1',
  'split': 'test',
  'n_samples': 500}}

In [72]:
for super_weight in sw:
    print(super_weight)
    ppl_res = session.analyzer.metrics_analyzer.measure_perplexity_impact(super_weight, n_samples=500)
    print(f"Perplexity impact for {super_weight}: {ppl_res}")

Layer 1 mlp.down_proj.weight[1764, 1710]


2025-06-17 17:38:35,116 - SuperWeightManager_13406116624 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-17 17:39:40,413 - SuperWeightManager_13406116624 - INFO - Restored 1/1 weights


Perplexity impact for Layer 1 mlp.down_proj.weight[1764, 1710]: {'super_weight': SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00), 'baseline_perplexity': 17.51461410522461, 'modified_perplexity': 9591.20703125, 'perplexity_ratio': 547.6116672412978, 'perplexity_increase': 9573.692417144775, 'impact_severity': 'catastrophic', 'dataset_info': {'name': 'wikitext', 'config': 'wikitext-2-raw-v1', 'split': 'test', 'n_samples': 500}}
Layer 15 mlp.down_proj.weight[1764, 6840]


2025-06-17 17:40:51,588 - SuperWeightManager_13406116624 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-17 17:41:56,711 - SuperWeightManager_13406116624 - INFO - Restored 1/1 weights


Perplexity impact for Layer 15 mlp.down_proj.weight[1764, 6840]: {'super_weight': SuperWeight(layer=15, coords=[1764, 6840], input=-283.00, output=415.00), 'baseline_perplexity': 17.51461410522461, 'modified_perplexity': 17.874475479125977, 'perplexity_ratio': 1.0205463489940108, 'perplexity_increase': 0.3598613739013672, 'impact_severity': 'minimal', 'dataset_info': {'name': 'wikitext', 'config': 'wikitext-2-raw-v1', 'split': 'test', 'n_samples': 500}}


In [17]:
accuracy_impact = session.analyzer.metrics_analyzer.measure_accuracy_impact(sophisticated_sw, n_samples=100)

README.md:   0%|          | 0.00/6.84k [00:00<?, ?B/s]

hellaswag.py:   0%|          | 0.00/4.36k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/2.53k [00:00<?, ?B/s]

2025-06-10 01:49:32,746 - SuperWeightManager_13442570496 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-10 01:49:55,441 - SuperWeightManager_13442570496 - INFO - Restored 1/1 weights


In [18]:
accuracy_impact

{'super_weight': SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00),
 'task': 'hellaswag',
 'baseline_accuracy': 0.57,
 'modified_accuracy': 0.37,
 'accuracy_drop': 0.19999999999999996,
 'accuracy_ratio': 0.6491228070175439,
 'impact_severity': 'moderate',
 'n_samples': 100}

In [19]:
for super_weight in sw:
    print(super_weight)
    acc_res = session.analyzer.metrics_analyzer.measure_accuracy_impact(super_weight, n_samples=200)
    print(f"Accuracy impact for {super_weight}: {acc_res}")

Layer 1 mlp.down_proj.weight[1764, 1710]


2025-06-10 01:51:27,451 - SuperWeightManager_13442570496 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-10 01:52:12,462 - SuperWeightManager_13442570496 - INFO - Restored 1/1 weights


Accuracy impact for Layer 1 mlp.down_proj.weight[1764, 1710]: {'super_weight': SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00), 'task': 'hellaswag', 'baseline_accuracy': 0.475, 'modified_accuracy': 0.355, 'accuracy_drop': 0.12, 'accuracy_ratio': 0.7473684210526316, 'impact_severity': 'moderate', 'n_samples': 200}
Layer 3 mlp.down_proj.weight[1764, 1902]


2025-06-10 01:52:59,266 - SuperWeightManager_13442570496 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-10 01:53:44,296 - SuperWeightManager_13442570496 - INFO - Restored 1/1 weights


Accuracy impact for Layer 3 mlp.down_proj.weight[1764, 1902]: {'super_weight': SuperWeight(layer=3, coords=[1764, 1902], input=-63.56, output=-64.69), 'task': 'hellaswag', 'baseline_accuracy': 0.475, 'modified_accuracy': 0.465, 'accuracy_drop': 0.009999999999999953, 'accuracy_ratio': 0.9789473684210527, 'impact_severity': 'minimal', 'n_samples': 200}
Layer 15 mlp.down_proj.weight[1764, 6840]


2025-06-10 01:54:30,827 - SuperWeightManager_13442570496 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-10 01:55:15,710 - SuperWeightManager_13442570496 - INFO - Restored 1/1 weights


Accuracy impact for Layer 15 mlp.down_proj.weight[1764, 6840]: {'super_weight': SuperWeight(layer=15, coords=[1764, 6840], input=-283.00, output=415.00), 'task': 'hellaswag', 'baseline_accuracy': 0.475, 'modified_accuracy': 0.47, 'accuracy_drop': 0.0050000000000000044, 'accuracy_ratio': 0.9894736842105263, 'impact_severity': 'minimal', 'n_samples': 200}


In [27]:
session.model

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
    (rotary_emb): OlmoRotaryEmbedding()
  )
  (lm_head): 

## Super Activation Source Analysis

In [76]:
trace_result = session.analyzer.trace_activation_source(sw[0])

# Print the key findings
print("🔍 Activation Source Analysis:")
print(f"Pathway: {trace_result['pathway_analysis']['super_weight_location']}")
print(f"Key findings: {trace_result['pathway_analysis']['key_findings']}")
print(f"Total amplification: {trace_result['computational_flow']['amplification_factors']['total_amplification']:.1f}x")

# See which tokens contribute most
print("\n📍 Top Contributing Tokens:")
for contrib in trace_result['token_contributions'][:5]:
    print(f"  {contrib['token']}: {contrib['activation_value']:.1f}")

2025-06-17 18:01:32,736 - SuperWeightAnalyzer_13406117104 - INFO - Tracing activation source for Layer 1 mlp.down_proj.weight[1764, 1710]
2025-06-17 18:01:34,246 - SuperWeightAnalyzer_13406117104 - INFO - DOWN PROJ INPUT channel 1710: max_val = 401.50


🔍 Activation Source Analysis:
Pathway: Layer 1, Channel 1710
Key findings: ['Massive amplification detected: 415.9x increase', 'Extremely large activation magnitude: 401.5', 'Gate and up projections contribute roughly equally']
Total amplification: 415.9x

📍 Top Contributing Tokens:
  Apple: -401.5
   company: -0.0
   tech: 0.0
   worldwide: 0.0
   Inc: 0.0


In [77]:
trace_result

{'super_weight': SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00),
 'input_text': 'Apple Inc. is a worldwide tech company.',
 'tokens': ['Apple',
  ' Inc',
  '.',
  ' is',
  ' a',
  ' worldwide',
  ' tech',
  ' company',
  '.'],
 'pathway_analysis': {'super_weight_location': 'Layer 1, Channel 1710',
  'pathway_status': 'complete',
  'key_findings': ['Massive amplification detected: 415.9x increase',
   'Extremely large activation magnitude: 401.5',
   'Gate and up projections contribute roughly equally'],
  'activation_magnitude_progression': {'input_magnitude': 0.96533203125,
   'output_magnitude': 401.5,
   'total_amplification_factor': 415.91906929691453}},
 'token_contributions': [{'position': 0,
   'token': 'Apple',
   'activation_value': -401.5,
   'absolute_activation': 401.5,
   'is_max_contributor': True},
  {'position': 7,
   'token': ' company',
   'activation_value': -0.00027060508728027344,
   'absolute_activation': 0.00027060508728027344,
   'is_ma

In [80]:
# trace activation backwards
backward_trase_result = session.analyzer.trace_activation_backwards(sw[0])

2025-06-17 18:11:50,404 - SuperWeightAnalyzer_13406117104 - INFO - Tracing backwards to find source of super activation for Layer 1 mlp.down_proj.weight[1764, 1710]
2025-06-17 18:11:51,313 - SuperWeightAnalyzer_13406117104 - INFO - 🎯 Layer 0 final output channel 1710: max = 0.047


In [81]:
backward_trase_result

{'super_weight': SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00),
 'input_text': 'Apple Inc. is a worldwide tech company.',
 'tokens': ['Apple',
  ' Inc',
  '.',
  ' is',
  ' a',
  ' worldwide',
  ' tech',
  ' company',
  '.'],
 'source_analysis': {'progression_summary': [{'component': 'Input Embeddings',
    'max_magnitude': 0.0076751708984375,
    'max_position': 5},
   {'component': 'Layer 0 Attention Output',
    'max_magnitude': 0.0266571044921875,
    'max_position': 7},
   {'component': 'Layer 0 MLP Output',
    'max_magnitude': 0.048309326171875,
    'max_position': 6},
   {'component': 'Layer 0 Final Output (→ Layer 1 Input)',
    'max_magnitude': 0.04656982421875,
    'max_position': 6},
   {'component': 'Layer 1 Attention Output',
    'max_magnitude': 0.050628662109375,
    'max_position': 0}],
  'major_amplification_steps': [],
  'source_identification': 'Input Embeddings',
  'key_insights': ['Total amplification from source: 6.6x']},
 'component_co

In [22]:
sw[0]

SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00)

## Finding strategies for zero activations

In [23]:
mathematical_analysis = session.analyzer.mathematical_super_activation_analysis(sw[0])

2025-06-26 19:30:09,938 - SuperWeightAnalyzer_13509141712 - INFO - Mathematical super activation analysis for Layer 1 mlp.down_proj.weight[1764, 1710]
2025-06-26 19:30:09,939 - SuperActivationAnalyzer_13509141808 - INFO - Super activation analysis for Layer 1 mlp.down_proj.weight[1764, 1710]
2025-06-26 19:30:09,941 - SuperActivationAnalyzer_13509141808 - INFO - Detected architecture: MLPArchitectureInfo(type=gated_mlp, activation=silu, components=['gate', 'up', 'down'])


In [24]:
mathematical_analysis['mathematical_breakdown']

{'input_analysis': {'input_vector_shape': [2048],
  'input_magnitude': 45.21875,
  'input_mean': 4.470348358154297e-06,
  'input_std': 0.99951171875},
 'step1_gate_projection': {'operation': 'gate_output = W_gate @ x',
  'bias_term': None,
  'result_target_channel': 15.2421875,
  'has_bias': False},
 'step2_activation': {'operation': 'activated_gate = SILU(gate_output)',
  'input_to_activation': 15.2421875,
  'activation_output_target_channel': 15.2421875,
  'activation_function': 'silu',
  'activation_derivative_at_point': 1.000003395600431},
 'step3_up_projection': {'operation': 'up_output = W_up @ x',
  'bias_term': None,
  'result_target_channel': -26.34375,
  'has_bias': False},
 'step4_hadamard_product': {'operation': 'hadamard_result = activated_gate ⊙ up_output (ELEMENT-WISE MULTIPLICATION)',
  'activated_gate_component': 15.2421875,
  'up_output_component': -26.34375,
  'super_activation_result': -401.5,
  'formula': 'y[:, 1710] = SILU(W_gate[1710] @ x) * (W_up[1710] @ x)',
  

In [10]:
mathematical_analysis['weight_analysis']

{'target_channel_info': {'channel_index': 1710,
  'description': 'Channel 1710 in intermediate space where super activation occurs'},
 'gate_weights_analysis': {'operation': 'gate_output[1710] = W_gate[1710] @ x',
  'weight_vector_norm': 1.39453125,
  'max_weight': 0.317138671875,
  'min_weight': -0.2432861328125,
  'mean_weight': 0.0005812644958496094,
  'std_weight': 0.0308074951171875,
  'bias_term': None,
  'top_5_positive_weights': [{'dimension': 803, 'weight': 0.317138671875},
   {'dimension': 1542, 'weight': 0.266357421875},
   {'dimension': 1930, 'weight': 0.246337890625},
   {'dimension': 1344, 'weight': 0.2396240234375},
   {'dimension': 517, 'weight': 0.1917724609375}],
  'top_5_negative_weights': [{'dimension': 1981, 'weight': -0.2432861328125},
   {'dimension': 623, 'weight': -0.2041015625},
   {'dimension': 1916, 'weight': -0.149658203125},
   {'dimension': 967, 'weight': -0.1490478515625},
   {'dimension': 130, 'weight': -0.1474609375}]},
 'up_weights_analysis': {'operat

In [11]:
mathematical_analysis['attack_vectors']

{'zero_gate_attack': {'attack_name': 'Zero Gate Projection Attack',
  'current_output': 15.2421875,
  'required_change': -15.2421875,
  'modification_strategy': {'modify_dimension': 803,
   'current_value': 1.09765625,
   'required_new_value': -46.963929585257894,
   'input_change': -48.061585835257894},
  'feasibility': 'medium'},
 'zero_up_attack': {'attack_name': 'Zero Up Projection Attack',
  'current_output': -26.34375,
  'required_change': 26.34375,
  'modification_strategy': {'modify_dimension': 1764,
   'current_value': -9.7890625,
   'required_new_value': 85.87051196808511,
   'input_change': 95.65957446808511},
  'feasibility': 'medium'},
 'activation_saturation_attack': {'attack_name': 'SiLU Saturation Attack',
  'current_output': 15.2421875,
  'saturation_target': -10.0,
  'required_change': -25.2421875,
  'expected_activation_output': -0.00045397868702434395,
  'modification_strategy': {'modify_dimension': 803,
   'current_value': 1.09765625,
   'required_new_value': -78.4

In [12]:
mathematical_analysis

{'super_weight': SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00),
 'architecture_info': MLPArchitectureInfo(type=gated_mlp, activation=silu, components=['gate', 'up', 'down']),
 'model_agnostic_analysis': True,
 'mathematical_breakdown': {'input_analysis': {'input_vector_shape': [2048],
   'input_magnitude': 45.21875,
   'input_mean': 4.470348358154297e-06,
   'input_std': 0.99951171875},
  'step1_gate_projection': {'operation': 'gate_output = W_gate @ x',
   'bias_term': None,
   'result_target_channel': 15.2421875,
   'has_bias': False},
  'step2_activation': {'operation': 'activated_gate = SILU(gate_output)',
   'input_to_activation': 15.2421875,
   'activation_output_target_channel': 15.2421875,
   'activation_function': 'silu',
   'activation_derivative_at_point': 1.000003395600431},
  'step3_up_projection': {'operation': 'up_output = W_up @ x',
   'bias_term': None,
   'result_target_channel': -26.34375,
   'has_bias': False},
  'step4_hadamard_product': 

In [9]:
sw[0].original_value

0.63232421875